In [ ]:
#identify which querys are question querys
#delete them from the list but keep information
#rank the querys with bm25
#rank the question queries seperately:
    #gpt generates sub querys
    #bm25.search() for each sub query
    #aggregate score
    #update original query score

#always keep track of information qid, docid, docno etc. 

In [1]:
#imports
import openai
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [2]:
client = openai.OpenAI() #connect to OpenAI API

In [3]:
def ask_gpt(prompt, model="gpt-4", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

In [4]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [5]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [6]:
querys = pt_dataset.get_topics('query')

In [8]:
answers = dict()
for i in range(len(querys)):
    determine_abbrevation = f""" 
    You are an scientific expert especially in the domain of Information Retrieval. Your task is to detect whether
    a given query, which is given as a text below delimited by triple quotes, is a question. You can detect whether a query
    is a question if it contains question words like 'What' or 'Why'.
    For example given a query 'What is crx' you should answer yes, since the query contains 'What' which is a 
    question word. However if the given query is 'Information Retrieval' you should answer no, since there is no
    question word in the query.

    query: '''{querys['query'][i]}'''
    """
    answer = ask_gpt(prompt=determine_abbrevation) #check answer more carefully perhaps model will return not only {yes,no}
    #print(answer)
    qid = str(querys['qid'][i])
    if "yes" in answer.lower().strip():
        answers[qid] = True
    else:
        answers[qid] = False
print(answers)

{'1': False, '2': False, '3': False, '4': False, '5': False, '6': False, '7': False, '8': False, '9': False, '10': False, '11': False, '12': False, '13': False, '14': False, '15': False, '16': False, '17': False, '19': False, '20': False, '21': False, '22': False, '23': False, '24': False, '25': False, '26': False, '27': False, '28': False, '29': False, '30': False, '31': False, '32': False, '33': False, '34': False, '35': False, '36': False, '37': False, '38': False, '39': False, '40': False, '41': False, '42': False, '43': False, '44': False, '45': False, '46': False, '47': False, '48': False, '49': False, '50': False, '51': False, '52': False, '53': False, '54': False, '55': False, '56': True, '57': False, '58': False, '59': True, '60': True, '61': False, '62': True, '63': False, '64': False, '65': False, '66': False, '67': False, '68': False, '18': False}


In [9]:
questions = list()
for key in answers.keys():
    if bool(answers[key]):
        #find query
        for i in range(len(querys)):
            if querys['qid'][i] == key:
                query= querys['query'][i]
                print("Old query: ",query)
        #ask gpt to expand query
        expand = f""" 
        You are an expert in Information Retrieval.  I am building an retrieval System specificially for the scientific domain, just like Google Scholar I want to find scientific papers for a query,. I want to improve the precision of my retrieval system by expanding specific question querys. Please understand that the information need behind a question query is much different than for a keyword query. For example the intent behind a question query like "What is Deep Learning" much more likely is to find papers that focus on an introduction to deep learning and explains core concepts. Meanwhile the intent behind the keyword query "Deep Learning" is find any papers that focus on deep learning.
        Your task is to semanticly interpret a question query, which is below delimited by triple quotes, and return three to six expanded queries so I can gather the score for each query and then aggregate over the scores to calculate a final score. Here are some examples:
        For the question query 'What is deep learning' the intent is to find papers that introduce and explain deep learning. At the end of each query please add a '$'.
        Therefore expanded querys could be:
        'Introduction to deep Learning$'
        'Overview of deep Learning$'
        'basic concepts of deep learning$'
        For the question query 'Why use boolean retrieval model' the inent is to find papers that gve insights to reasons to apply the boolean retrieval model. Therefore expanded querys could be:
        'Reasons to use boolean retrieval model$'
        'Introduction to boolean retrieval model$'

        query: '''{query}'''
        """
        new_query = ask_gpt(prompt=expand).lower().strip().replace("'", " ").replace('"', ' ')
        print("New query: ",new_query)
        questions.append({query: new_query})

Old query:  what makes natural language processing natural
New query:   understanding natural language processing$ 
 principles of natural language processing$ 
 introduction to natural language processing$ 
 fundamentals of natural language processing$ 
 concepts behind natural language processing$ 
Old query:  what is ahp
New query:   introduction to ahp$ 
 overview of ahp$ 
 basic concepts of ahp$ 
 understanding ahp$ 
 principles of ahp$ 
Old query:  what is information retrieval
New query:   introduction to information retrieval$ 
 overview of information retrieval$ 
 basic concepts of information retrieval$ 
 principles of information retrieval$ 
 understanding information retrieval$ 
Old query:  how to avoid spam results
New query:   preventing spam results$ 
 methods to filter spam results$ 
 guide to avoid spam in search results$ 
 strategies to eliminate spam results$ 
 understanding spam results and how to avoid them$ 


In [10]:
print(questions)

[{'what makes natural language processing natural': ' understanding natural language processing$ \n principles of natural language processing$ \n introduction to natural language processing$ \n fundamentals of natural language processing$ \n concepts behind natural language processing$ '}, {'what is ahp': ' introduction to ahp$ \n overview of ahp$ \n basic concepts of ahp$ \n understanding ahp$ \n principles of ahp$ '}, {'what is information retrieval': ' introduction to information retrieval$ \n overview of information retrieval$ \n basic concepts of information retrieval$ \n principles of information retrieval$ \n understanding information retrieval$ '}, {'how to avoid spam results': ' preventing spam results$ \n methods to filter spam results$ \n guide to avoid spam in search results$ \n strategies to eliminate spam results$ \n understanding spam results and how to avoid them$ '}]


In [11]:
split_q = list()
for query in questions:
    for key in query:
        query = query[key].replace('\n', ' ').strip()
        split_q.append(query.split('$'))
print(split_q)

[['understanding natural language processing', '   principles of natural language processing', '   introduction to natural language processing', '   fundamentals of natural language processing', '   concepts behind natural language processing', ''], ['introduction to ahp', '   overview of ahp', '   basic concepts of ahp', '   understanding ahp', '   principles of ahp', ''], ['introduction to information retrieval', '   overview of information retrieval', '   basic concepts of information retrieval', '   principles of information retrieval', '   understanding information retrieval', ''], ['preventing spam results', '   methods to filter spam results', '   guide to avoid spam in search results', '   strategies to eliminate spam results', '   understanding spam results and how to avoid them', '']]


now do the ranking


In [12]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [13]:
sub_queries = split_q[0]

In [60]:
import pandas as pd
def score_documents(subqueries, retriever, qid, original_query):
    final_scores = {}

    for subquery in subqueries:
        results = retriever.search(subquery)
        for index, row in results.iterrows():
            doc_id = row['docno']
            score = row['score']
            docno = row['docno']
            
            if doc_id in final_scores:
                final_scores[doc_id]['score'] += score
            else:
                final_scores[doc_id] = {'score': score, 'docno': docno}

    # Convert the final scores to a list of dictionaries
    results_list = [
        {'qid': qid, 'docid': doc_id, 'docno': details['docno'], 'score': details['score'], 'query': original_query}
        for doc_id, details in final_scores.items()
    ]

    # Create a DataFrame from the results list
    results_df = pd.DataFrame(results_list).sort_values(by='score', ascending=False)

    return results_df

In [68]:
run = score_documents(sub_queries, bm25, 1, "test")

/usr/local/lib/python3.10/dist-packages/pyterrier/batchretrieve.py:307: UserWarning: Skipping empty query for qid 1
  warn("Skipping empty query for qid %s" % qid)


In [69]:
print(run.head(3))

     qid     docid     docno      score query
14     1  J86-2001  J86-2001  73.413873  test
302    1  J95-3006  J95-3006  69.230891  test
484    1  N03-5001  N03-5001  69.113129  test


In [76]:
run.loc[1] = [1,'abc','321', 73.0, 'test']
print(run.at[1, 'docid'])

abc


In [47]:
run.at[1, 'score'] = 0
print(run)

  qid   docid                               docno  rank      score  \
1   1  125137   1989.ipm_journal-ir0volumeA25A4.2     1   0.000000   
2   1  125817  2005.ipm_journal-ir0volumeA41A5.11     2  14.144223   

                                      query  
1  retrieval system improving effectiveness  
2  retrieval system improving effectiveness  


In [ ]:
#iterate through querys and determine which is a question query
#perform normal bm25 ranking for each query except question querys
#rank question querys seperately:
    #generate sub querys with gpt
    #score each sub query with bm25.search()
    #aggregate scores of each sub query to final score for original query
#add ranking rows to bm25 

In [86]:
run2 = bm25(querys)
run3 = run2[:3]


run3.loc[4]= [1,918366899762, 'abc', 'a', 17, 'test']
print("LENGHT",len(run3))
print(run3)

LENGHT 4
  qid         docid                               docno rank      score  \
0   1         94858        2004.cikm_conference-2004.47    0  15.681777   
1   1        125137   1989.ipm_journal-ir0volumeA25A4.2    1  15.047380   
2   1        125817  2005.ipm_journal-ir0volumeA41A5.11    2  14.144223   
4   1  918366899762                                 abc    a  17.000000   

                                      query  
0  retrieval system improving effectiveness  
1  retrieval system improving effectiveness  
2  retrieval system improving effectiveness  
4                                      test  


/tmp/ipykernel_20054/2019941890.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run3.loc[4]= [1,918366899762, 'abc', 'a', 17, 'test']
